# emTools

### a Python module to interact with SerialEM to enable automated Transmission Electron Microscopy

## Table of contents
1. [Installation instructions](#installation)
  - How to use this jupyter notebook
  - Install necessary dependencies in Python
  - [install/update emTools](#install1)

2. [Getting started](#howtos)
  - [Tutorial 1: Sort a Navigator](#tutorial1)


## Installation instructions<a name="installation"></a>

### Prepare your PC for the use of Python and emTools

- first we will install Python and then use it to fetch the packages needed for emTools to work.
- Install Anaconda from [here](https://www.anaconda.com/download/ "Download Anaconda"). EmTools has been tested to work with both Python 2 and 3. Choose the appropriate OS.
- Start the Anaconda [Navigator](http://docs.anaconda.com/anaconda/user-guide/getting-started/ "Getting started with Anaconda").
- as explained in [section 4](http://docs.anaconda.com/anaconda/user-guide/getting-started/#run-python-in-a-jupyter-notebook "Run Python in a Jupyter Notebook"), install and start a Jupyter notebook.
- Download this [tutorial file](https://git.embl.de/schorb/serialem_tools/raw/master/emTools.ipynb) (use "save link as..."), open it in your Jupyter session and follow the instructions.

----
### How to use this jupyter notebook

In this document there are text blocks and code blocks that can be edited and executed.
In order to run a code block, click into it to activate and then press `CTRL+ Enter`. There is a Python logo at the top right of the notebook that indicates whether there are still operations running.

### Install necessary dependencies in Python

The following code block uses Anaconda to fetch necessary packages. Start the code by pressing `CTRL+ Enter` inside the code block.

In [ ]:
# this will load the package needed for python package management
import conda.cli

# this will install the dependencies
conda.cli.main('conda', 'install',  '-y', 'numpy', 'scipy', 'pandas', 'scikit-image', 'git', 'pip')
conda.cli.main('conda', 'install',  '-y', '-c conda-forge', 'tifffile')

## install/update emTools <a name="install1"></a>

Now we can install emTools using pip, another package manager for Python. Start by pressing `CTRL+ Enter` inside the code block. 

To update to the current version, simply execut this code block anytime.

In [ ]:
# this will load the pip package needed for python package management
from pip._internal import main as pipmain

# this will install the current version of emTools
pipmain(['install','git+https://git.embl.de/schorb/serialem_tools'])

----



## Getting started <a name="howtos"></a>



### Tutorial 1: Sort a navigator file<a name="tutorial1"></a>

Have a look at the files in the directory ["`tutorials/Tutorial1-sorting`"](https://git.embl.de/schorb/serialem_tools/tree/master/tutorials/Tutorial1-sorting). If you open the navigator file `nav1.nav` in SerialEM, you will get the following:

#### the Navigator
![The navigator](images/nav1_1.png)

If you double-click the `Map` item, you will see this:
#### the Map
![The cells](images/nav1.png)

We have a series of points for each cell. However they are not listed such that all points lying within one cell are acquired in a row.

Let's use emTools to create a sorted Navigator file.


#### Reading in the Navigator

First, we will import the emTools module and thus makei its functions available for the Python script.
Then we will read in the Navigator file.



In [ ]:
# import the emTools module and make its functions available
import emtools as em

# load the navigator file
navfile = 'tutorials/Tutorial1-sorting/nav1.nav'
navlines = em.loadtext(navfile)

# now we have a list of strings for each row of the file, let's look at the first 10.
print('first 10 lines of the navigator: \n')
print(navlines[0:10])
print('\n----\n')


# let's convert them into a list of dictionaries with one entry for each navigator Item
allitems = em.fullnav(navlines)
# Let look at one item
print('Item number 7 from the navigator file: \n')
print(allitems[6])
print('\n----\n')

# Each entry for this item is associated with a dedicated key. For example the Stage Coordinates
print('Stage coordinates of item number 4: \n')
print(allitems[3]['StageXYZ'])
print('\n----\n')





#### Now, let's find out how to order a navigator list using the function `ordernav`.




In [ ]:
newnav = em.ordernav(allitems)

print('Let\'s look at the first points in the list...')
print(newnav[1]['# Item'])
print(newnav[2]['# Item'])
print(newnav[3]['# Item'])
print(newnav[4]['# Item'])
print(newnav[5]['# Item'])
print(newnav[6]['# Item'])
print(newnav[7]['# Item'])

# and now we need to write the ordered navigator file.

# create new file by copying the header of the input file
newnavf = navfile[:-4] + '_sorted.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n" % navlines[1])

# fill the new file   
for nitem in newnav: 
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

# done

----

and this is what we get...

![The sorted navigator](images/nav1_sorted.png)


----

#### Batch process parameters of navigator items

Let's now also try and see if we can change some of the general parameters of these navigator items, like for example z-height, color or the 'Acquire' flag.

In [ ]:
# create new file by copying the header of the input file
newnavf = navfile[:-4] + '_playground.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n"% navlines[1])

# create a new list to collect ouput navigator items.
outnav=list()

# add the map. This we don't want to modify
outnav.append(newnav[0])

# Then modify and add the rest...

for item in newnav[1:] :
    item['StageXYZ'][2] = '31.415'
    item['Color'] = ['3']
    item['Acquire'] = ['1']
    outnav.append(item)

# fill the new file   
for nitem in outnav: 
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

# done

----

and we get...

![The modified navigator](images/nav1_playground.png)


yellow points at the new height that are set to be acquired.

----